import csv
from collections import defaultdict

toJson = open('anime.json', 'w')
count = 0
toJson.write('[')
with open('anime.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        if row[0] == 'anime_id':
            continue
        
        
        genre_list = row[2].split(',')
        genres = '['
        for g in range(len(genre_list)):
            genre = genre_list[g].replace(" ", "")
            genres += '"'
            genres += genre
            if g != len(genre_list)-1:
                genres += '",'
            else:
                genres += '"]'
                
        rating = row[5]
        if rating == '':
            rating = -1
        
        if (count != 0):
            toJson.write(',\n')
        
        toJson.write('{ "anime_id" : "' + str(row[0]) + '", "name" : "' + str(row[1]) + '", "genre" : '
              + str(genres) + ', "type" : "' + str(row[3]) + '", "episodes" : "'
              + str(row[4]) + '", "rating" : ' + str(rating) + ', "members" : ' + str(row[6]) + '}')
        
        count += 1
    
toJson.write(']')
toJson.close()

In [12]:
# import data

import json

rating = json.load(open('rating_200k.json', encoding='utf8'))
animedata = json.load(open('anime.json', encoding='utf8'))


In [13]:
# Make a train / validation set

# Filter rating of '-1' in rating dataset

rating_filtered = []
for entry in rating:
    if (entry['rating'] != '-1'):
        rating_filtered.append(entry)

In [14]:
# Approach 2 (scrambled dataset)

import random

def scrambled(orig):
    dest = orig[:]
    random.shuffle(dest)
    return dest

rating_scrambled = scrambled(rating_filtered)

In [15]:
# Approach 3 (evenly distributed dataset)

def build_rating_dataset():

    rating_toModify = rating_filtered[:]

    current_user_id = 1
    rating_3 = []

    checknumber = 150000

    while(len(rating_toModify) > 3000):

        for j in range(0,len(rating_toModify)-1):

            if j >= len(rating_toModify)-1:
                break

            this_rating = rating_toModify[j]
            current_user_id_str = str(current_user_id)

            if this_rating['user_id'] == current_user_id_str:

                rating_3.append(this_rating)
                rating_toModify.pop(j)
                current_user_id += 1

            if int(this_rating['user_id']) > current_user_id:
                current_user_id += 1



        if current_user_id >= 2012:
            current_user_id = 1

    remaining = rating_toModify[:]
    rating_3 = rating_3 + remaining
    rating_3 = rating_3[:150000]
    
    return rating_3

In [16]:
rating_3 = build_rating_dataset()

In [109]:
# separate train/validation by half

data_size = 70000
training_set = rating_3[:data_size]
validation_set = rating_3[data_size:data_size*2]
test_set = rating_3[data_size*2:]

In [110]:
# constructing latent factor model

beta_u = defaultdict(float)
beta_i = defaultdict(float)
alpha = defaultdict(float)


# This function constructs beta_u, beta_i, alpha for latent factor model

def construct_latent_factor_model():

    rating_by_user = defaultdict(list)    # list of animes with rating that each user watched
    rating_by_anime = defaultdict(list)    # list of dictionary with anime as keys, and containing user and ratings
    
    for entry in training_set:

        anime_dict = {}
        if (entry['rating'] != '-1'):
            anime_dict['anime_id'] = entry['anime_id']
            anime_dict['rating'] = entry['rating']
            rating_by_user[entry['user_id']].append(anime_dict)

        user_dict = {}
        if (entry['rating'] != '-1'):
            user_dict['rating'] = entry['rating']
            user_dict['user_id'] = entry['user_id'] 
            rating_by_anime[entry['anime_id']].append(user_dict)
            

            
    # calculating average ratings by user        
    
    average_rating_by_user = defaultdict(float)
    
    for user in rating_by_user:

        animes = rating_by_user[user]
        total_rating = 0
        count = 0

        for anime in animes:    
            total_rating += float(anime['rating'])
            count += 1

        if count != 0:
            average_rating_by_user[user] = total_rating / count
            

    # constructing beta_u
    
    total_user_rating = 0
    user_count = 0
    
    for user in average_rating_by_user:
        total_user_rating += average_rating_by_user[user]
        user_count += 1

    total_average_rating_byuser = total_user_rating / user_count

    for user in average_rating_by_user:
        beta_u[user] = average_rating_by_user[user] - total_average_rating_byuser
        
        

    # Calculating average rating by anime
    
    anime_average_rating = defaultdict(float)      # average rating of each anime

    for entry in animedata:
        if (entry['rating'] != -1 and entry['rating'] != 0):
            anime_average_rating[entry['anime_id']] = entry['rating']
            
            
    # Calculating beta_i

    total_anime_rating = 0
    anime_count = 0
    for anime in anime_average_rating:
        total_anime_rating += anime_average_rating[anime]
        anime_count += 1

    total_average_rating_byanime = total_anime_rating / anime_count

    for anime in anime_average_rating:
        beta_i[anime] = anime_average_rating[anime] - total_average_rating_byanime
      
    
    # alpha
    
    alpha = anime_average_rating 
    


In [112]:
reviewsPerUser = defaultdict(list)
reviewsPerAnime = defaultdict(list)
for r in training_set:
    reviewsPerUser[r['user_id']].append(r)
    reviewsPerAnime[r['anime_id']].append(r)

In [113]:
trainRatings = [int(r['rating']) for r in training_set]
globalAverage = sum(trainRatings) * 1.0 / len(trainRatings)

validMSE = 0
for r in validation_set:
    se = (int(r['rating']) - globalAverage)**2
    validMSE += se

validMSE /= len(validation_set)

print("Validation MSE (average only) = " + str(validMSE))

Validation MSE (average only) = 2.332054253877099


In [135]:
betaU = {}
betaI = {}
for u in reviewsPerUser:
    betaU[u] = 0

for i in reviewsPerAnime:
    betaI[i] = 0

alpha = globalAverage

def resetBeta():
    global betaU
    global betaI
    global alpha
    betaU = {}
    betaI = {}
    for u in reviewsPerUser:
        betaU[u] = 0

    for i in reviewsPerAnime:
        betaI[i] = 0

    alpha = globalAverage

def iterate(lamb):
    newAlpha = 0
    for r in training_set:
        newAlpha += int(r['rating']) - (betaU[r['user_id']] + betaI[r['anime_id']])
    alpha = newAlpha / len(training_set)
    for u in reviewsPerUser:
        newBetaU = 0
        for r in reviewsPerUser[u]:
            newBetaU += int(r['rating']) - (alpha + betaI[r['anime_id']])
        betaU[u] = newBetaU / (lamb + len(reviewsPerUser[u]))
    for i in reviewsPerAnime:
        newBetaI = 0
        for r in reviewsPerAnime[i]:
            newBetaI += int(r['rating']) - (alpha + betaU[r['user_id']])
        betaI[i] = newBetaI / (lamb + len(reviewsPerAnime[i]))
    mse = 0
    for r in training_set:
        prediction = alpha + betaU[r['user_id']] + betaI[r['anime_id']]
        mse += (int(r['rating']) - prediction)**2
    regularizer = 0
    for u in betaU:
        regularizer += betaU[u]**2
    for i in betaI:
        regularizer += betaI[i]**2
    mse /= len(training_set)
    return mse, mse + lamb*regularizer

In [136]:
# Fit with lambda = 1
resetBeta()
mse,objective = iterate(1)
newMSE,newObjective = iterate(1)
iterations = 2

In [137]:
newMSE

1.2850353201867877

In [138]:
while iterations < 10 or objective - newObjective > 0.0001:
    mse, objective = newMSE, newObjective
    newMSE, newObjective = iterate(1)
    iterations += 1
    print("Objective after " + str(iterations) + " iterations = " + str(newObjective))
    print("MSE after " + str(iterations) + " iterations = " + str(newMSE))

Objective after 3 iterations = 3096.0561687081877
MSE after 3 iterations = 1.2819922007956852
Objective after 4 iterations = 3086.39999410803
MSE after 4 iterations = 1.2814807155769579
Objective after 5 iterations = 3067.397715665469
MSE after 5 iterations = 1.281273315669468
Objective after 6 iterations = 3047.79334374342
MSE after 6 iterations = 1.2811259643557653
Objective after 7 iterations = 3029.4463805945174
MSE after 7 iterations = 1.2809977920928621
Objective after 8 iterations = 3012.656885845118
MSE after 8 iterations = 1.2808800977957493
Objective after 9 iterations = 2997.3812087609385
MSE after 9 iterations = 1.28077059644888
Objective after 10 iterations = 2983.5081164014746
MSE after 10 iterations = 1.2806684015270227
Objective after 11 iterations = 2970.920527115425
MSE after 11 iterations = 1.2805729483998365
Objective after 12 iterations = 2959.5083207548814
MSE after 12 iterations = 1.2804837606069497
Objective after 13 iterations = 2949.170222156615
MSE after 13 i

In [133]:
def calcValidMSE():
    validMSE = 0
    for r in validation_set:
        bu = 0
        bi = 0
        if r['user_id'] in betaU:
            bu = betaU[r['user_id']]
        if r['anime_id'] in betaI:
            bi = betaI[r['anime_id']]
        prediction = alpha + bu + bi
        validMSE += (int(r['rating']) - prediction)**2

    validMSE /= len(validation_set)
    print("Validation MSE = " + str(validMSE))
    return validMSE

In [132]:
calcValidMSE()

defaultdict(<class 'float'>, {})


TypeError: unsupported operand type(s) for +: 'collections.defaultdict' and 'float'

In [95]:
betaUs = [(betaU[u], u) for u in betaU]
betaIs = [(betaI[i], i) for i in betaI]
betaUs.sort()
betaIs.sort()

print("Maximum betaU = " + str(betaUs[-1][1]) + ' (' + str(betaUs[-1][0]))
print("Maximum betaI = " + str(betaIs[-1][1]) + ' (' + str(betaIs[-1][0]))

Maximum betaU = 355 (2.3891606829652003
Maximum betaI = 1657 (2.5913361219054227


In [101]:
def lambdaIteration(lambd, newMSE, newObjective):
    iterations = 1
    while (iterations < 10 or objective - newObjective > 0.0001) and not iterations > 100:
        mse, objective = newMSE, newObjective
        newMSE, newObjective = iterate(lambd)
        iterations += 1
        #print("Objective after " + str(iterations) + " iterations = " + str(newObjective))
        #print("MSE after " + str(iterations) + " iterations = " + str(newMSE))

In [149]:
validMSE = 0
for r in validation_set:
    bu = 0
    bi = 0
    if r['user_id'] in betaU:
        bu = betaU[r['user_id']]
    if r['anime_id'] in betaI:
        bi = betaI[r['anime_id']]
    prediction = alpha + bu + bi
    validMSE += (int(r['rating']) - prediction)**2

validMSE /= len(validation_set)
print("Validation MSE = " + str(validMSE))

Validation MSE = 1.607817393727215


In [197]:
lambdas = [0.01, 0.1, 0.5, 0.75, 0.875, 1, 1.25, 1.5, 1.75, 2, 5, 10, 25, 50, 100]
best_lambda = 0
best_mse = 100000
lambdaResult = []
for lamb in lambdas:
    resetBeta()
    newMSE,newObjective = iterate(lamb)
    lambdaIteration(lamb, newMSE, newObjective)
    mse = calcValidMSE()
    lambdaResult.append((lamb, mse))
    if mse < best_mse:
        best_mse = mse
        best_lambda = lamb

Validation MSE = 1.609951074270031
Validation MSE = 1.632536446731577
Validation MSE = 1.6692205392838553
Validation MSE = 1.6463113508721245
Validation MSE = 1.6380219498512807
Validation MSE = 1.6300056531377174
Validation MSE = 1.6186865382291677
Validation MSE = 1.6089054168000279
Validation MSE = 1.602443300681581
Validation MSE = 1.5974148789221603
Validation MSE = 1.5814022585722542
Validation MSE = 1.607817393727215
Validation MSE = 1.6818387645334538
Validation MSE = 1.7801640611938188
Validation MSE = 1.909824365129494


In [200]:
for(a, b) in lambdaResult:
    print(b)

1.609951074270031
1.632536446731577
1.6692205392838553
1.6463113508721245
1.6380219498512807
1.6300056531377174
1.6186865382291677
1.6089054168000279
1.602443300681581
1.5974148789221603
1.5814022585722542
1.607817393727215
1.6818387645334538
1.7801640611938188
1.909824365129494


In [186]:
resetBeta()
newMSE,newObjective = iterate(best_lambda)
lambdaIteration(best_lambda, newMSE, newObjective)

In [151]:
testMSE = 0
for r in test_set:
    bu = 0
    bi = 0
    if r['user_id'] in betaU:
        bu = betaU[r['user_id']]
    if r['anime_id'] in betaI:
        bi = betaI[r['anime_id']]
    prediction = alpha + bu + bi
    testMSE += (int(r['rating']) - prediction)**2

testMSE /= len(test_set)
print("Test MSE = " + str(testMSE))

Test MSE = 1.810081348394134


In [154]:
trainingMSE = 0
for r in training_set:
    bu = 0
    bi = 0
    if r['user_id'] in betaU:
        bu = betaU[r['user_id']]
    if r['anime_id'] in betaI:
        bi = betaI[r['anime_id']]
    prediction = alpha + bu + bi
    testMSE += (int(r['rating']) - prediction)**2

trainingMSE /= len(training_set)
print("Training MSE = " + str(trainingMSE))

Training MSE = 0.0


In [188]:
# feature for latent factor model

def feature(r, threshhold):
    a = alpha
    bu = 0
    bi = 0
    if r['user_id'] in betaU:
        bu = betaU[r['user_id']]
    if r['anime_id'] in betaI:
        bi = betaI[r['anime_id']]
    prediction = alpha + bu + bi
    
    feat = [1]
    feat.append(prediction)
    #feat.append(jaccard_sim_user(r['user_id'],r['anime_id']))
    feat.append(1 if jaccard_sim_user(r['user_id'],r['anime_id']) > threshhold else 0)
    return feat
        
        

In [189]:
# call function to build anime_user_dict and user_anime_dict

anime_users_dict = get_anime_users()
user_anime_dict = get_user_anime()

In [190]:
# X = [feature_train(anime) for anime in beta_i]
# y = [float(d['rating']) for d in training_set]
resetBeta()
newMSE,newObjective = iterate(best_lambda)
lambdaIteration(best_lambda, newMSE, newObjective)

X_train = [feature(d) for d in training_set]
y_train = [float(d['rating']) for d in training_set]

X_validation = [feature(d) for d in validation_set]
y_validation = [float(d['rating']) for d in validation_set]

X_test = [feature(d) for d in test_set]
y_test = [float(d['rating']) for d in test_set]

TypeError: feature() missing 1 required positional argument: 'threshhold'

In [ ]:
import numpy

theta, residuals, rank, s = numpy.linalg.lstsq(X_train,y_train)

In [191]:
import math

def mse(X, y, theta, size):
    MSSum = 0.0
    for i in range(0,size):
        MSSum += numpy.square(int(y[i]) - numpy.dot(X[i], theta))
    return MSSum / size

In [192]:
def trainAndValidate(threshold):
    X_train = [feature(d, threshold) for d in training_set]
    y_train = [float(d['rating']) for d in training_set] 
    
    X_validation = [feature(d, threshold) for d in validation_set]
    y_validation = [float(d['rating']) for d in validation_set]
    valMSE = mse(X_validation,y_validation, theta, len(X_validation))
    return valMSE

In [195]:
best_thresh = 0
best_err = 1000
thresholds = [0.0075 * i for i in range(10)]
thresholdResults = []
for thresh in thresholds:
    error = trainAndValidate(thresh)
    print("Threshold: " + str(thresh) + " Error: "+ str(error))
    thresholdResults.append((thresh, error))
    if error < best_err:
        best_err = error
        best_thresh = thresh

print(best_thresh)
print(best_err)

Threshold: 0.0 Error: 1.50887304386
Threshold: 0.0075 Error: 1.50887304386
Threshold: 0.015 Error: 1.50868440541
Threshold: 0.0225 Error: 1.50810613212
Threshold: 0.03 Error: 1.508058339
Threshold: 0.0375 Error: 1.50832911652
Threshold: 0.045 Error: 1.50871356807
Threshold: 0.0525 Error: 1.50909127208
Threshold: 0.06 Error: 1.50931522378
Threshold: 0.0675 Error: 1.51064524351
0.03
1.508058339


In [196]:
thresholdResults

[(0.0, 1.5088730438611639),
 (0.0075, 1.5088730438611639),
 (0.015, 1.5086844054061941),
 (0.0225, 1.5081061321185254),
 (0.03, 1.5080583389976012),
 (0.0375, 1.5083291165244339),
 (0.045, 1.5087135680694026),
 (0.0525, 1.5090912720770477),
 (0.06, 1.5093152237787846),
 (0.0675, 1.5106452435097131)]

In [177]:
# Calculating MSE and variance

print("Validation MSE is " + str(mse(X_validation,y_validation, theta, len(X_validation))))
print("Training MSE is " + str(mse(X_train,y_train, theta, len(X_train))))
print("Test MSE is " + str(mse(X_test,y_test, theta, len(X_test))))
print("variance is " + str(numpy.var([int(d) for d in y_validation])))

print("\nRMSE is "+ str(math.sqrt(mse(X_validation,y_validation, theta, len(X_validation)))))

Validation MSE is 1.50945589526
Training MSE is 1.33095689358
Test MSE is 1.6556327443
variance is 2.26928253061

RMSE is 1.2285991597169903


In [ ]:
# Collaborative Filtering + Regression for rating prediction
# Collaborative Filtering for watch prediction (in case we can't figure out how to do rating prediction)
# Basic model for latent factor model
# How to optimize

### Graph
# rating according to genre
# rating according to types (Movie/OVA/TV Series)
# rating according to number of members
# Most popular word occuring in title
# Average number of members by genres


In [ ]:
# Collaborative Filtering + Regression for rating prediction
# Collaborative Filtering for watch prediction (in case we can't figure out how to do rating prediction)
# Basic model for latent factor model
# How to optimize

In [155]:
# Rating prediction based on collaborative filtering + regression


def get_anime_users():
    #first get a dictionary of key: business user: set
    anime_user_dict = {}
    for review in training_set:
        anime = review['anime_id']
        user = review['user_id']
        if anime in anime_user_dict:
            user_set = anime_user_dict[anime]
            user_set.add(user)
        else:
            user_set = set()
            user_set.add(user)
            anime_user_dict[anime] = user_set
    return anime_user_dict

def get_user_anime():
    #first get a dictionary of key: business user: set
    user_anime_dict = {}
    for review in training_set:
        anime = review['anime_id']
        user = review['user_id']
        if user in user_anime_dict:
            anime_set = user_anime_dict[user]
            anime_set.add(anime)
        else:
            anime_set = set()
            anime_set.add(anime)
            user_anime_dict[user] = anime_set
    return user_anime_dict



def jaccard_sim_user(check_user, check_anime):
    #given a business, find the most similar business
    max_sim = 0.0
#     count = 0

    if check_anime not in anime_users_dict: return False
    else:
        users = anime_users_dict[check_anime]
        for user in users:
            if user == check_user: continue
            if check_user not in user_anime_dict: continue
            current_user_anime = user_anime_dict[user]
            check_user_anime = user_anime_dict[check_user]
            intersect = current_user_anime.intersection(check_user_anime)
            union = current_user_anime.union(check_user_anime)
            # print(intersect)
            # print(union)
            similarity = float(len(intersect))/len(union)
            if similarity > max_sim:
                max_sim = similarity
            #max_sim += similarity
#             count += 1

#      print(max_sim)
    return max_sim

#     if (max_sim > 0.059):
#         return True
#     else:
#         return False




In [31]:
# call function to build anime_user_dict and user_anime_dict

anime_users_dict = get_anime_users()
user_anime_dict = get_user_anime()

In [32]:
import numpy

# feature based on collaborative filtering
def feature(u,a):
    feat = [1]
    #feat.append(1 if jaccard_sim_user(u,a) > 0.01 else 0)
    feat.append(jaccard_sim_user(u,a))
#     if (similarity(u,a)):
#         feat.append(1)
#     else:
#         feat.append(0)
    return feat

In [33]:
X_train = [feature(d['user_id'],d['anime_id']) for d in training_set]
y_train = [float(d['rating']) for d in training_set]
X_validation = [feature(d['user_id'],d['anime_id']) for d in validation_set]
y_validation = [float(d['rating']) for d in validation_set]
theta, residuals, rank, s = numpy.linalg.lstsq(X_train,y_train)

In [34]:
theta

array([ 7.70948912,  1.53688363])

In [35]:
import math

def mse(X, y, theta, size):
    MSSum = 0.0
    for i in range(0,size):
        MSSum += numpy.square(int(y[i]) - numpy.dot(X[i], theta))
    return MSSum / size

In [36]:
# Calculating MSE and variance

print("Validation MSE is " + str(mse(X_validation,y_validation, theta, len(X_validation))))
print("Training MSE is " + str(mse(X_train,y_train, theta, len(X_train))))
print("variance is " + str(numpy.var([int(d) for d in y_validation])))

print("\nRMSE is "+ str(math.sqrt(mse(X_validation,y_validation, theta, len(X_validation)))))

Validation MSE is 2.29699162412
Training MSE is 2.2622388855
variance is 2.30704351556

RMSE is 1.5155829321147043


In [37]:
# Watch prediction based on collaborative filtering


import random

anime_id_set = set()
user_id_set = set()
for l in rating_3:
    anime_id_set.add(l['anime_id'])
    user_id_set.add(l['user_id'])
    
visited_pair = []
for l in validation_set:
    visited_pair.append((l['user_id'], l['anime_id']))
anime_id_list = list(anime_id_set)
user_id_list = list(user_id_set)
visited_pair = set(visited_pair)




# Build random pairs 

random_pair = []
count = 0

while(count < 75000):
    randomIndex1 = random.randint(0, len(anime_id_set)-1)
    randomIndex2 = random.randint(0, len(user_id_set)-1)
    userid = user_id_list[randomIndex2]
    animeid = anime_id_list[randomIndex1]
    
    if (userid,animeid) not in visited_pair:
        temp_dict = {}
        temp_dict['user_id'] = userid
        temp_dict['anime_id'] = animeid
    
        random_pair.append(temp_dict)
        count += 1
        
        
# Combine random pairs with validation set to make complete validation set

complete_validation_set = validation_set + random_pair

In [38]:
# Watch prediction

def watch_prediction():
    
    true_pos = 0
    true_neg = 0
    
    for l in validation_set:
        u,a = l['user_id'], l['anime_id']
        if jaccard_sim_user(u,a):
            true_pos += 1
            
    for l in random_pair:
        u,a = l['user_id'], l['anime_id']
        if not jaccard_sim_user(u,a):
            true_neg += 1
            
    accuracy = (true_pos + true_neg) / len(complete_validation_set)
    print("Accuracy is ", accuracy)
    


In [39]:
watch_prediction()

Accuracy is  0.65856


In [40]:
# To-do List

# Finish latent factor model with regresison
# Try using both jaccard similarity and latent factor model
# Writeup
